In [1]:
from pycognito.aws_srp import AWSSRP
from getpass import getpass
from pprint import pprint

import boto3
import pandas as pd
import json
import requests

BASE_URL="https://api.carta.contextualize.us.com"

# Create a CSV file for dashboards

In [2]:
data = pd.read_excel("kinematics_summary.xlsx", None) 
df = pd.concat(data)
df["machine"] = df.index.get_level_values(0)
print(df.head())
print(df.tail())
df.reset_index(drop=True, inplace=True)
df.to_csv("Laser Data.csv")

            Layer  time on  time off  time exposition  distance on  \
renishaw 0      1  1.90473   1.06816          2.97280     2.272890   
         1      2  1.90924   1.50222          3.41146     2.267675   
         2      3  1.90472   1.07359          2.97831     2.278525   
         3      4  1.90924   1.48980          3.39904     2.267721   
         4      5  1.90472   1.07173          2.97645     2.278555   

            distance off  total distance  avg speed on  avg speed off  \
renishaw 0      0.655434        2.933990      1.193287       0.613610   
         1      0.738088        3.005763      1.187737       0.491331   
         2      0.682365        2.960889      1.196252       0.635592   
         3      0.675910        2.943630      1.187761       0.453692   
         4      0.673262        2.951816      1.196268       0.628201   

            avg speed   machine  
renishaw 0   0.986945  renishaw  
         1   0.881078  renishaw  
         2   0.994151  renishaw  
    

# Upload file to Carta

Note: Files sent to the `carta-dashboards` directory will be selectable in Carta

## Authenticate

In [10]:
username = getpass("Enter your Carta username: ")

Enter your Carta username:  ········


In [11]:
password = getpass("Enter your Carta password: ")

Enter your Carta password:  ········


In [12]:
response = requests.get(
    url=f"{BASE_URL}/auth"
)
userPool = json.loads(response.content)
poolId = userPool["userPoolId"]
region = userPool["region"]
clientId = userPool["userPoolWebClientId"]

client = boto3.client("cognito-idp", region_name=region)
aws = AWSSRP(username=username, 
             password=password, 
             pool_id=poolId,
             client_id=clientId, 
             client=client)
tokens = aws.authenticate_user()
idToken = tokens['AuthenticationResult']['IdToken']

## Upload the file

In [13]:
upload_file = open("Laser Data.csv", "rb")
file_stream = {'file':("carta-dashboards/Laser Data.csv",upload_file.read())} # Note that the file goes into carta-dashboards
response = requests.post(
    url=f"{BASE_URL}/files/Carta",
    files=file_stream,
    headers={"Authorization": "Bearer {}".format(idToken)},    
    stream=True
)
upload_file.close()
print(f"Response {response.status_code}: {response.reason}")
pprint(json.loads(response.text))

Response 200: OK
{'dateCreated': '2023-11-15T14:36:19.0473829+00:00',
 'dateModified': '2023-11-15T14:36:19.0473829+00:00',
 'id': '0cc5e9b4-ad74-2e6a-fbf5-5ff1b93199f6',
 'name': 'Laser Data.csv',
 'owner': 'blopez',
 'path': 'carta-dashboards/Laser Data.csv',
 'source': 'Carta',
 'type': 'File'}


## Grant permissions

Allow other UTEP users to view this file.

In [14]:
fileId=json.loads(response.text)['id']
response = requests.put(
    url=f"{BASE_URL}/permissions/{fileId}/UTEP:All?userType=userGroup&role=guest",
    headers={"Authorization": "Bearer {}".format(idToken)}
)
print(f"Response {response.status_code}: {response.reason}")
pprint(json.loads(response.text))

Response 200: OK
'Read'


Allow Bryant's user to edit the file, in case he wants to update the file in future:

In [15]:
response = requests.put(
    url=f"{BASE_URL}/permissions/{fileId}/blopez?userType=user&role=editor",
    headers={"Authorization": "Bearer {}".format(idToken)}
)
print(f"Response {response.status_code}: {response.reason}")
pprint(json.loads(response.text))

Response 200: OK
'Execute, Write, Read, Clone'


## Update the file

If you want to make changes to the file, do the following: First get a listing of all dashboard files, in order to get the file identifier of the file that needs updating. Then, update the file through a `PATCH` request. 

In [16]:
response = requests.get(
    url=f"{BASE_URL}/files/Carta/list",
    params = {"path" : "/carta-dashboards/"},
    headers={"Authorization": "Bearer {}".format(idToken)}
)
print(f"Response {response.status_code}: {response.reason}")
pprint(json.loads(response.text))

Response 200: OK
[{'dateCreated': '2023-08-11T23:11:30.3092581+00:00',
  'dateModified': '2023-09-26T20:02:23.2971109+00:00',
  'id': '7ac4f36f-4305-3c5c-72c0-6f32fb2ba80b',
  'name': 'Global Test Artifact Data Exchange Program.csv',
  'owner': 'admin.utep',
  'path': '/carta-dashboards/Global Test Artifact Data Exchange Program.csv',
  'source': 'Carta',
  'type': 'File'},
 {'dateCreated': '2023-11-15T14:36:19.0473829+00:00',
  'dateModified': '2023-11-15T14:36:19.0473829+00:00',
  'id': '0cc5e9b4-ad74-2e6a-fbf5-5ff1b93199f6',
  'name': 'Laser Data.csv',
  'owner': 'blopez',
  'path': '/carta-dashboards/Laser Data.csv',
  'source': 'Carta',
  'type': 'File'},
 {'dateCreated': '2023-08-14T21:17:31.8075689+00:00',
  'dateModified': '2023-08-14T21:57:27.8137749+00:00',
  'id': '54c4faf4-a090-ba1c-f366-aa627894f4f6',
  'name': 'Laser Data.csv',
  'owner': 'admin.utep',
  'path': '/carta-dashboards/Laser Data.csv',
  'source': 'Carta',
  'type': 'File'}]


In [17]:
update_file = open("Laser Data.csv", "rb")
file_stream = {'file':("carta-dashboards/Laser Data.csv",update_file.read())} 
response = requests.patch(
    url=f"{BASE_URL}/files/Carta/file/{fileId}",
    files=file_stream,
    headers={"Authorization": "Bearer {}".format(idToken)},    
    stream=True
)
update_file.close()
print(f"Response {response.status_code}: {response.reason}")
pprint(json.loads(response.text))

Response 200: OK
{'dateCreated': '2023-11-15T14:36:19.0473829+00:00',
 'dateModified': '2023-11-15T14:37:19.9445532+00:00',
 'id': '0cc5e9b4-ad74-2e6a-fbf5-5ff1b93199f6',
 'name': 'Laser Data.csv',
 'owner': 'blopez',
 'path': '/carta-dashboards/Laser Data.csv',
 'source': 'Carta',
 'type': 'File'}
